In [ ]:
import re
import numpy as np
from scripts.scrape import get_data, get_test_data



# Day 15

In [ ]:
data = get_data(15)
test = get_test_data(15)
data

In [ ]:
data = [int(d) for d in data[0].split(',')]

In [ ]:
seq = data.copy()

last = {}
for i in range(len(seq)):
    last[seq[i]] = [i, i]
for i in range(len(seq), 2020):
    lookup = seq[i-1]
    val = last[lookup][1] - last[lookup][0]
    seq.append(val)
    if val in last.keys():
        last[val] = [last[val][1], i]
    else: 
        last[val] = [i, i]        
seq[-1]

In [ ]:
# Part 2

seq = data.copy()

last = {}
for i in range(len(seq)):
    last[seq[i]] = [i, i]
for i in range(len(seq), 30000000):
    lookup = seq[i-1]
    val = last[lookup][1] - last[lookup][0]
    seq.append(val)
    if val in last.keys():
        last[val] = [last[val][1], i]
    else: 
        last[val] = [i, i]        

seq[-1]

# Day 16

In [ ]:
data = get_data(16,'\n\n')
test = get_test_data(16)
data

In [ ]:
data = [d.split('\n') for d in data]

In [ ]:
validation = data[0]
my_ticket = data[1][1:]
nearby_tickets = data[2][1:]

In [ ]:
def validate(number, validation):
    validation_ranges = [r for v in validation for r in re.findall("([0-9]+)-([0-9]+)",v)]
#     print(validation_ranges)
    valid = [(number >= int(v[0])) & (number <= int(v[1])) for v in validation_ranges]
#     print(valid)
    if np.any(valid):
        return(0)
    else: return(number)
    

In [ ]:
np.sum([validate(int(n), validation) for t in nearby_tickets for n in t.split(',')])

### Part 2

In [ ]:
def validate(number, validation):
    validation_ranges = [r for v in validation for r in re.findall("([0-9]+)-([0-9]+)",v)]
    valid = [(number >= int(v[0])) & (number <= int(v[1])) for v in validation_ranges]
    if np.any(valid):
        return(True)
    else: return(False)
    

In [ ]:
mask = [np.all([validate(int(n), validation) for n in t.split(',')]) for t in nearby_tickets]
nearby_tickets = np.array([[int(n) for n in t.split(',')] for t in nearby_tickets])
nearby_tickets = nearby_tickets[mask]
nearby_tickets

In [ ]:
def guess(number):
    validation_ranges = [r for v in validation for r in re.findall("([0-9]+)-([0-9]+)",v)]
    valid = [(number >= int(v[0])) & (number <= int(v[1])) for v in validation_ranges]
    return(np.array(valid)[np.arange(1, 40, 2)] + np.array(valid)[np.arange(0, 39, 2)])
guess(337)

In [ ]:
truth_values = np.apply_along_axis(guess, 1, nearby_tickets)
logic_matrix = np.array([np.apply_along_axis(np.all, 0, truth_values[:,i,:]) for i in range(20)])
assignment = {}
for i in range(20):
    row = np.where(logic_matrix.sum(1) == 1)[0][0]
    column = np.where(logic_matrix[row, :])[0][0]
    assignment[column] = row
    logic_matrix[:, column] = 0
np.prod([int(my_ticket[0].split(',')[v]) for v, k in assignment.items() if k in range(6)])    

In [ ]:
def check_ticket(ticket):
    validation_ranges = [r for v in validation for r in re.findall("([0-9]+)-([0-9]+)",v)]
    valid = [(int(validation_ranges[2*assignment[idx]][0]) <= item <=  int(validation_ranges[2*assignment[idx]][1])) + \
        (int(validation_ranges[2*assignment[idx] + 1][0]) <= item <=  int(validation_ranges[2*assignment[idx]+ 1][1])) for idx, item in enumerate(ticket)]
    return(np.all(valid))
np.all([check_ticket(t) for t in nearby_tickets])

# Day 17

In [ ]:
data = get_data(16)
test = get_test_data(16)
data